In [2]:
import pandas as pd
import pickle

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import glm

In [4]:
from sklearn.preprocessing import StandardScaler


In [5]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [6]:
#finalgen_samples = pd.read_csv('../final_gen.csv')['sample_name']
first_gen = pd.read_csv('../key_files/generation_1_sample_names.txt',header=None)[0]
samples = first_gen.to_list()

#clim_sites_during_exp = pd.read_csv('/carnegie/nobackup/scratch/tbellagio/grene/data/bio')
clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_sites_era5_year_2018.csv')

sites_af = pd.Series(samples).str.split('_').str[0].astype(int)

sites_af.name = 'site'

In [7]:
unseen_sites = [7, 50, 56, 21, 41, 47]

# Convert the list to a Series and concatenate with the existing Series
new_series = pd.Series(unseen_sites, name='site')

In [8]:
sites_af = pd.concat([sites_af, new_series], ignore_index=True)

In [9]:
sites_af

0       1
1       1
2       1
3       1
4       1
       ..
327    50
328    56
329    21
330    41
331    47
Name: site, Length: 332, dtype: int64

In [199]:
env = sites_af.reset_index().merge(clim_sites_during_exp).drop(['index'],axis=1)

In [200]:
#env = env.drop_duplicates()

In [201]:
env_variable = env['bio1']

In [202]:
# Standardize the environmental variable
scaler = StandardScaler()
env_variable_scaled = scaler.fit_transform(env_variable.values.reshape(-1, 1))

In [204]:
pd.DataFrame(env_variable_scaled).to_csv('env.csv',index=None)

In [207]:
env_site_scaled = pd.concat([env['site'],pd.Series(env_variable_scaled.flatten())],axis=1)

In [208]:
env_site_scaled.columns = ['site', 'env_scaled']

In [209]:
env_site_scaled

,site,env_scaled
0,1,-0.779581
1,1,-0.779581
2,1,-0.779581
3,1,-0.779581
4,1,-0.779581
...,...,...
327,50,-1.101596
328,56,-1.778135
329,21,-0.715968
330,41,-0.642677


In [210]:
env_site_scaled.to_csv('env_site_scaled.csv',index=None)

In [13]:
import os

In [14]:
files = os.listdir('../baypass_first_gen/individual_gfiles/')

In [15]:
partitions = [int(file.split('_')[1].replace('.txt', '')) for file in files if '.txt' in file]

In [16]:
partitions.sort()

In [116]:
partitions[-1]

201

In [117]:
len(partitions)

202

In [118]:
pwd -P

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go'

In [119]:
import random
import subprocess

In [154]:
samples = pd.read_csv('../key_files/merged_sample_table.csv')

In [155]:
samples = samples[samples['generation'] ==1]

In [156]:
samples = samples['sample_name'].to_list()

In [158]:
import pickle

def create_fake_cv(samples, unseen_sites):
    """
    Creates a fake cross-validation file where all samples are used for training
    and a single fake sample name is used for each unseen site in the test set.

    Parameters:
    samples (list): Original list of cross-validation splits.
    unseen_sites (list): List of unseen site numbers.

    Returns:
    list: New list of cross-validation splits with fake samples for unseen sites.
    """
    # Flatten the original sample list to get all unique samples
    #all_samples = sorted(set([sample for split in samples for sample in split[0]]))
    
    # Create the new fake splits
    fake_splits = []
    for site in unseen_sites:
        train = samples  # Use all samples for training
        # Create one fake sample name for each unseen site
        test = [f"{site}_1_1"]
        # Append the new split as a tuple of (train, test)
        fake_splits.append((train, test))
    
    return fake_splits


In [160]:
# Unseen site numbers
unseen_sites = [7, 50, 56, 21, 41, 47]

# Create fake cross-validation splits
fake_cv_splits = create_fake_cv(samples, unseen_sites)

# Save the fake splits to a new pickle file
output_path = '../jacknife_first_gen/fake_splits_samples_for_unseen_sites.pkl'
with open(output_path, 'wb') as file:
    pickle.dump(fake_cv_splits, file)

print(f"Fake cross-validation splits saved to {output_path}")

Fake cross-validation splits saved to ../jacknife_first_gen/fake_splits_samples_for_unseen_sites.pkl


In [18]:
file_path = '../jacknife_first_gen/fake_splits_samples_for_unseen_sites.pkl'

# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples_fake = pickle.load(file)

In [19]:
len(samples_fake)

6

In [165]:
len(samples_fake[0][0])

326

In [167]:
samples_fake[0][1]

['7_1_1']

In [168]:
samples_fake[1][1]

['50_1_1']

In [170]:
# Loop through splits 0 to 31
for split in range(len(samples)):
    # Define the folder path
    folder_path = f'results_sites_unseen/split_{split}'
    # Create the directory (makedirs allows creating intermediate directories if they don't exist)
    os.makedirs(folder_path, exist_ok=True)

In [171]:
pwd

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go'

In [22]:
len(samples_fake)

6

In [23]:
for split in range(len(samples_fake)):
    print(split)

0
1
2
3
4
5


In [223]:
# create sbatch files to submit on cedar server
shfiles = []
for split in [6]: #range(len(samples_fake)):
    for partition in partitions:
        seed = random.randint(1,100000000)
        file = f'shfiles/partition_{partition}_{split}.sh'
        cmd = f'python run_partition_binomial_reg_first_gen_unseen_sites.py {partition} {split}'
        text = f'''#!/bin/bash
#SBATCH --job-name=run_partition_binomial_reg{partition}_{split}
#SBATCH --time=1:00:00  # Time limit set to 4 hours
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=30gb
#SBATCH --output=run_partition_binomial_reg_{partition}_{split}_%j.out
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

module load python/3.11_conda
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake
export LD_LIBRARY_PATH="/home/tbellagio/miniforge3/envs/run_baypass/lib:$LD_LIBRARY_PATH"
cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/binomial_regression_firstgen_go
{cmd}

'''
        with open(file, 'w') as o:
            o.write("%s" % text)
        shfiles.append(file)

In [225]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 64725


CompletedProcess(args=['sbatch', 'shfiles/partition_0_0.sh'], returncode=0)

In [226]:
## now run the shfiles
for shfile in shfiles:
    # Submit each sbatch script to the SLURM scheduler
    subprocess.run(["sbatch", shfile], check=True)

Submitted batch job 64726
Submitted batch job 64727
Submitted batch job 64728
Submitted batch job 64729
Submitted batch job 64730
Submitted batch job 64731
Submitted batch job 64732
Submitted batch job 64733
Submitted batch job 64734
Submitted batch job 64735
Submitted batch job 64736
Submitted batch job 64737
Submitted batch job 64738
Submitted batch job 64739
Submitted batch job 64740
Submitted batch job 64741
Submitted batch job 64742
Submitted batch job 64743
Submitted batch job 64744
Submitted batch job 64745
Submitted batch job 64746
Submitted batch job 64747
Submitted batch job 64748
Submitted batch job 64749
Submitted batch job 64750
Submitted batch job 64751
Submitted batch job 64752
Submitted batch job 64753
Submitted batch job 64754
Submitted batch job 64755
Submitted batch job 64756
Submitted batch job 64757
Submitted batch job 64758
Submitted batch job 64759
Submitted batch job 64760
Submitted batch job 64761
Submitted batch job 64762
Submitted batch job 64763
Submitted ba

In [11]:
files = os.listdir('../baypass_lastgen/individual_gfiles_last_gen/')
loci_names = [file for file in files if 'loci' in file]

In [95]:
for split in range(len(samples)):
    print(split)
    print(len(os.listdir(f'results/split_{split}')))

0
203
1
202
2
202
3
202
4
202
5
202
6
202
7
202
8
202
9
202
10
202
11
202
12
202
13
202
14
202
15
202
16
202
17
202
18
202
19
202
20
202
21
202
22
202
23
202
24
202
25
202
26
202
27
202
28
202
29
202
30
202


In [25]:
for split in range(len(samples_fake)):
    print(split)
    partitions_r = {}
    for i in range(len(partitions)):  # /baypass_first_gen/individual_gfiles/
        pickle_file_path = f'../baypass_first_gen/individual_gfiles/loci_partition_{i}'
        with open(pickle_file_path, 'rb') as file:
            loci_f = pickle.load(file)
        results = pd.read_csv(f'results_sites_unseen/split_{split}/partition{i}.csv')
        results['snp_id'] = loci_f
        partitions_r[i] = results
    results = pd.concat(partitions_r).reset_index(drop=True)
    results.to_csv(f'results_sites_unseen/split_{split}/binomial_reg_results_first_gen_sites_unseen.csv',index=None)

0
1
2
3
4
5


In [30]:
pd.read_csv(f'results_sites_unseen/split_{split}/binomial_reg_results_first_gen_sites_unseen.csv')

,47,slope,pvalue,intercept,snp_id
0,0.271129,0.002634,8.171630e-01,-0.988079,1_346
1,0.075197,0.074634,8.585182e-05,-2.486135,1_353
2,0.097985,-0.022623,1.856125e-01,-2.226889,1_363
3,0.076062,0.377930,7.818571e-97,-2.378932,1_395
4,0.076062,0.377930,7.818571e-97,-2.378932,1_396
...,...,...,...,...,...
1054569,0.812567,-0.206419,6.118108e-59,1.402234,5_26975078
1054570,0.077901,0.056934,2.389608e-03,-2.453414,5_26975121
1054571,0.077901,0.056934,2.389608e-03,-2.453414,5_26975148
1054572,0.462689,0.007647,4.516182e-01,-0.147132,5_26975272
